In [1]:
from pyspark.sql import SparkSession

In [2]:
app = SparkSession.builder \
      .appName("Sales Data Analysis with integrating Gen AI") \
      .config("spark.jars","/path/to/mysql-connector-j-8.4.0.jar") \
      .getOrCreate()

In [3]:
import pandas as pd

In [4]:
import pandasai

In [5]:
from pandasai.llm.openai import OpenAI

In [6]:
import os

In [ ]:
os.environ['PANDASAI_API_KEY'] = ""

In [7]:
import os

from pandasai import SmartDataframe

In [10]:
import openai

In [11]:
from openai import OpenAI

In [ ]:
openai.api_key = ""

In [13]:
host_name = 'localhost'
port_number = 3306
database_name = "sales"
url = f"jdbc:mysql://{host_name}:{port_number}/{database_name}"

In [ ]:
connection_properties = {

    "user":"root",
    "password":"",
    "driver":"com.mysql.cj.jdbc.Driver"
}

In [15]:
table1 = "orders"

In [16]:
try:
    orders_df = app.read.jdbc(url=url, table=table1, properties=connection_properties)
    orders_df.show(n=10)
except Exception as e:
    print(f"error as {e}")

+------------+-------------------+-------------------+-------------------+-----------+-------------+--------------+------------+-------------+--------------+-------------+----------+-------------+--------+
| OrderNumber|          OrderDate|          Ship Date|Customer Name Index|    Channel|Currency Code|Warehouse Code|Region Index|Product Index|Order Quantity|Selling Price|Cost Price|Total Revenue|  Profit|
+------------+-------------------+-------------------+-------------------+-----------+-------------+--------------+------------+-------------+--------------+-------------+----------+-------------+--------+
| SO - 000225|2017-01-01 00:00:00|2017-01-13 00:00:00|                 28|  Wholesale|          NZD|        AXW291|          71|           11|             6|       2499.1|   1824.34|      14994.6| 4048.56|
|SO - 0003378|2017-01-01 00:00:00|2017-01-06 00:00:00|                  7|Distributor|          NZD|        AXW291|          54|            7|            11|       2351.7|   12

In [17]:
table2 = "customers"

In [18]:
try:
    customers_df = app.read.jdbc(url=url, table=table2, properties=connection_properties)
    customers_df.show()
except Exception as e:
    print(f"error as {e}")

+--------------+--------------+
|Customer Index|Customer Names|
+--------------+--------------+
|             1|     Avon Corp|
|             2|     WakeFern |
|             3|  Elorac, Corp|
|             4|     ETUDE Ltd|
|             5|  Procter Corp|
|             6| PEDIFIX, Corp|
|             7|       New Ltd|
|             8|  Medsep Group|
|             9|           Ei |
|            10|      21st Ltd|
|            11|    Apollo Ltd|
|            12|      Medline |
|            13|     Ole Group|
|            14|        Linde |
|            15| Rochester Ltd|
|            16|     3LAB, Ltd|
|            17|    Pure Group|
|            18| Eminence Corp|
|            19|    Qualitest |
|            20|   Pacific Ltd|
+--------------+--------------+
only showing top 20 rows



In [19]:
table3 = "products"

In [20]:
try:
   products_df = app.read.jdbc(url=url, table=table3, properties=connection_properties)
   products_df.show()
except Exception as e:
    print(f"error as {e}")

+-----+------------+
|Index|Product Name|
+-----+------------+
|    1|   Product 1|
|    2|   Product 2|
|    3|   Product 3|
|    4|   Product 4|
|    5|   Product 5|
|    6|   Product 6|
|    7|   Product 7|
|    8|   Product 8|
|    9|   Product 9|
|   10|  Product 10|
|   11|  Product 11|
|   12|  Product 12|
|   13|  Product 13|
|   14|  Product 14|
|   20|  Product 20|
+-----+------------+



In [21]:
table4 = "regions"

In [22]:
try:
    regions_df = app.read.jdbc(url=url, table=table4, properties=connection_properties)
    regions_df.show()
except Exception as e:
    print(f"error as {e}")

+-----+------------------+-----------------+--------+----------+----------+--------------------+
|Index|            Suburb|             City|postcode| Longitude|  Latitude|        Full Address|
+-----+------------------+-----------------+--------+----------+----------+--------------------+
|    1|      Freemans Bay|         Auckland|    1011|174.748652|-36.855732|Picton Street, Fr...|
|    2|         Nightcaps|        Southland|    9630|168.028823|  -45.9703|Lyne Street, Nigh...|
|    3|         Northcote|      North Shore|     627|174.755505|-36.804712|McBreen Avenue, N...|
|    4|          Bay View|           Napier|    4104|176.871662|-39.440389|Ferguson Street S...|
|    5|         Parklands|     Christchurch|    8083|172.705997|-43.472699|Forest Drive, Par...|
|    6|     Hamilton East|         Hamilton|    3216|175.305496|-37.781657|Rochford Court, H...|
|    7|          Te Kuiti|          Waitomo|    3910|175.163086|-38.330973|Alexandra Street,...|
|    8|           Opaheke|    

In [23]:
orders_df.createOrReplaceTempView("orders")

In [24]:
customers_df.createOrReplaceTempView("customers")

In [25]:
products_df.createOrReplaceTempView("products")

In [26]:
regions_df.createOrReplaceTempView("regions")

## find the month wise total profit

In [27]:
result1 = app.sql(
    '''
    select month(OrderDate) as `Month`,
    round(sum(profit), 2) as `Total Profit`
    from orders
    group by Month
    order by Month asc
    '''
)

result1.show()

+-----+------------+
|Month|Total Profit|
+-----+------------+
|    1|   4791277.3|
|    2|  4239747.77|
|    3|  5434826.12|
|    4|  4683583.31|
|    5|  4905461.72|
|    6|  4616685.11|
|    7|  4651450.05|
|    8|  4988199.63|
|    9|   5174513.3|
|   10|   5004587.3|
|   11|  4666011.03|
|   12|  4632769.78|
+-----+------------+



In [71]:
df = result1.toPandas()
df_smart = SmartDataframe(df)
df_smart.chat("""find which Month has highest Total Profit and which Month has lowest Total Profit 
              tell the month name like january february...""")

'The month with the highest total profit is March and the month with the lowest total profit is February.'

## --------------------------------------------------------------------------

## find the month over month profit

In [55]:
result2 = app.sql(
    '''
    with cte as
    (
    select month(OrderDate) as `Month`,
    round(sum(profit), 2) as `Total Profit`
    from orders
    group by Month
    order by Month asc
    )
    select *,
    round(sum(`Total Profit`) over(order by `Month` asc), 2) as `Month over Month Profit`
    from cte
    '''
)

result2.show()

+-----+------------+-----------------------+
|Month|Total Profit|Month over Month Profit|
+-----+------------+-----------------------+
|    1|   4791277.3|              4791277.3|
|    2|  4239747.77|             9031025.07|
|    3|  5434826.12|          1.446585119E7|
|    4|  4683583.31|           1.91494345E7|
|    5|  4905461.72|          2.405489622E7|
|    6|  4616685.11|          2.867158133E7|
|    7|  4651450.05|          3.332303138E7|
|    8|  4988199.63|          3.831123101E7|
|    9|   5174513.3|          4.348574431E7|
|   10|   5004587.3|          4.849033161E7|
|   11|  4666011.03|          5.315634264E7|
|   12|  4632769.78|          5.778911242E7|
+-----+------------+-----------------------+



## --------------------------------------------------------------------------

## top 10 products by profit

In [54]:
result3 = app.sql(
       '''
       with `profit_rank` as
       (
       select p.`Product Name`,o.`Order Quantity`, o.`Cost Price`,o.`Selling Price`, o.`Total Revenue`,
       round(o.`Order Quantity` * o.`Cost Price`, 2) as `Total Cost`,o.`Profit`,
       dense_rank() over(order by Profit desc) as `rank`
       from (orders as o) inner join (products as p)
       on o.`Product Index` = p.Index
       )
       select * from profit_rank where rank <=10
       '''
)
result3.show()

+------------+--------------+----------+-------------+-------------+----------+--------+----+
|Product Name|Order Quantity|Cost Price|Selling Price|Total Revenue|Total Cost|  Profit|rank|
+------------+--------------+----------+-------------+-------------+----------+--------+----+
|   Product 5|            12|   2749.28|       6545.9|      78550.8|  32991.36|45559.44|   1|
|   Product 1|            12|   2835.98|       6445.4|      77344.8|  34031.76|43313.04|   2|
|   Product 3|            12|   2903.45|       6452.1|      77425.2|   34841.4| 42583.8|   3|
|   Product 2|            12|   2555.11|       6083.6|      73003.2|  30661.32|42341.88|   4|
|   Product 2|            12|   2797.92|       6217.6|      74611.2|  33575.04|41036.16|   5|
|   Product 1|            11|   2590.42|       6318.1|      69499.1|  28494.62|41004.48|   6|
|   Product 3|            12|   2467.88|       5875.9|      70510.8|  29614.56|40896.24|   7|
|   Product 1|            12|   3138.82|       6539.2|      

In [58]:
df2 = result3.toPandas()
smart_df2 = SmartDataframe(df2)
smart_df2.chat("which product Name has the highest profit and which product Name has lowest profit")

'The product with the highest profit is Product 5 and the product with the lowest profit is Product 11.'

## --------------------------------------------------------------------------

## Total Sales and total profit for each products

In [72]:
result4 = app.sql(
       '''
       select p.`Product Name`, round(sum(o.`Total Revenue`), 2) as `Total Sales`,
       round(sum(o.`Profit`), 2) as `Total Profit`
       from (orders as o) inner join (products as p)
       on o.`Product Index` = p.Index
       group by `Product Name`
       order by `Total Profit` desc
       '''
)
result4.show()

+------------+------------+------------+
|Product Name| Total Sales|Total Profit|
+------------+------------+------------+
|   Product 1|2.54892991E7|  9596180.27|
|   Product 7|2.57101244E7|  9545304.86|
|   Product 2|2.28464975E7|  8705229.41|
|  Product 11|2.06218094E7|  7698384.32|
|   Product 5|1.70211557E7|  6445172.72|
|  Product 13|1.17746604E7|  4133532.73|
|   Product 9|   9261644.5|   3546673.2|
|  Product 14|   3144263.1|  1262405.75|
|   Product 6|   3341216.3|  1199190.73|
|  Product 10|   3114005.9|  1188981.28|
|   Product 8|   3215604.7|   1176884.6|
|  Product 12|   3109289.1|  1112039.98|
|   Product 4|   2857288.7|  1093602.43|
|   Product 3|   3066281.8|  1085530.14|
+------------+------------+------------+



In [73]:
df_new = result4.toPandas()
df_new_smart = SmartDataframe(df_new)
df_new_smart.chat("""which Product Name has the highest Total Profit 
                  and which Product Name has the lowest Total Profit.""")

'The Product with the highest Total Profit is Product 1, and the Product with the lowest Total Profit is Product 3.'

## --------------------------------------------------------------------------

## find the top 20 customers based on profit

In [74]:
result5 = app.sql(
       '''
       with `customer_sales` as
       (
       select c.`Customer Names`, round(sum(o.`Total Revenue`), 2) as `Total Sales`,
       round(sum(o.`Profit`), 2) as `Total Profit`
       from (orders as o) inner join (customers as c)
       on o.`Customer Name Index` = c.`Customer Index`
       group by `Customer Names`
       )
       select *,
       dense_rank() over(order by `Total Profit` desc) as `profit rank`
       from customer_sales
       '''
)
result5.show(n=2000)

+---------------+-----------+------------+-----------+
| Customer Names|Total Sales|Total Profit|profit rank|
+---------------+-----------+------------+-----------+
|       Medline |  4081485.9|  1506669.31|          1|
|     Pure Group|  3821592.9|  1402517.29|          2|
|        OUR Ltd|  3676035.4|  1398368.61|          3|
|     Apollo Ltd|  3600358.9|  1392406.16|          4|
|    OHTA'S Corp|  3533801.1|  1355516.68|          5|
|  Apotheca, Ltd|  3608492.7|  1337765.81|          6|
|  Eminence Corp|  3643426.5|   1332935.5|          7|
|          Ohio |  3347681.8|  1318352.27|          8|
|      ETUDE Ltd|  3288018.3|  1314732.71|          9|
|         Linde |  3314610.6|  1286557.86|         10|
|    Victory Ltd|  3407814.3|  1263720.89|         11|
|            Ei |  3273653.5|   1261536.3|         12|
|   Prasco Group|  3215933.0|  1231436.19|         13|
|      Ole Group|  3462881.6|  1227354.35|         14|
|  Rochester Ltd|  3039903.9|  1215109.37|         15|
|    Weime

In [76]:
df_new2 = result5.toPandas()
df_new2_smart = SmartDataframe(df_new2)
df_new2_smart.chat("""find which customer has the highest Total profit 
                   and which customer has the lowest Total profit """)

'The customer with the highest total profit is Medline , and the customer with the lowest total profit is WakeFern .'

## --------------------------------------------------------------------------

## find the yearly total profit

In [31]:
result6 = app.sql(
    ''' 
    select year(`OrderDate`) as `year`, round(sum(`Total Revenue`) , 2)as `Total Sales`, 
    round(sum(`Profit`) , 2) as `Total Profit`  
    from orders
    group by year
    order by year
    '''
)

result6.show()

+----+------------+-------------+
|year| Total Sales| Total Profit|
+----+------------+-------------+
|2017|5.25805347E7|1.967776122E7|
|2018|5.34636617E7|1.978917841E7|
|2019|4.85289442E7|1.832217279E7|
+----+------------+-------------+



In [42]:
df3 = result6.toPandas()
df3_smart = SmartDataframe(df3)
query = """find which year has the highest profit and which year has the lowest profit"""
df3_smart.chat(query) 

'The year with the highest profit is 2018.0 and the year with the lowest profit is 2019.0.'

## --------------------------------------------------------------------------

## find the quarter wise total profit

In [52]:
result7 = app.sql(
    '''  
    select quarter(OrderDate) as `quarter`, round(sum(`Total Revenue`), 2) as `Total Sales`,
    sum(`Profit`) as `Total Profit`
    from orders
    group by quarter
    '''
)

result7.show()

+-------+------------+--------------------+
|quarter| Total Sales|        Total Profit|
+-------+------------+--------------------+
|      1|3.91295879E7|1.4465851190000014E7|
|      3|3.98566853E7|1.4814162980000002E7|
|      4|3.78534724E7|1.4303368110000005E7|
|      2| 3.7733395E7|1.4205730140000006E7|
+-------+------------+--------------------+



In [62]:
df4 = result7.toPandas()
df4_smart = SmartDataframe(df4)
df4_smart.chat("""find which quarter has the highest total profit and which quarter has the lowest total profit.""")

'The quarter with the highest total profit is 3 and the quarter with the lowest total profit is 2.'

## --------------------------------------------------------------------------

## find the city wise total profit

In [65]:
result8 = app.sql(
    '''  
    select r.City, round(sum(o.`Total Revenue`), 2) as `Total Sales`,
    round(sum(o.`Profit`), 2) as `Total Profit`
    from (orders as `o`) inner join (regions as `r`)
    on o.`Region Index` = r.`Index`
    group by r.City
    '''
)

result8.show(n=1000)

+-----------------+------------+------------+
|             City| Total Sales|Total Profit|
+-----------------+------------+------------+
|       Wellington|   1591920.0|   576748.53|
|            Waipa|   1720560.0|   649829.57|
|         Franklin|   1020436.8|   400935.53|
|           Clutha|   3332291.9|  1345769.18|
|       Rangitikei|   1719199.9|   650441.19|
|     New Plymouth|   1280383.4|    425842.7|
| Palmerston North|   6967477.4|  2638189.85|
|     Christchurch|1.15266934E7|  4333779.49|
|Thames-Coromandel|   4377002.8|  1601213.88|
|         Auckland|   7463277.4|  2853433.75|
|        Waitakere|  1.119302E7|  4180236.23|
|         Gisborne|   1707005.9|   644008.32|
|    Central Otago|   1704493.4|   609966.44|
|          Porirua|   1321628.6|   523461.75|
|        Southland|   3106361.2|  1114282.64|
|          Rotorua|   1810916.2|   650169.93|
|          Hauraki|   1381781.2|   513584.58|
|     Kapiti Coast|   1513302.2|   585360.77|
|          Waikato|   1814808.9|  

In [69]:
df5 = result8.toPandas()
df5_smart = SmartDataframe(df5)
df5_smart.chat("""find which City has the highest total profit and which City has the lowest total profit""")

'The City with the highest total profit is Christchurch and the City with the lowest total profit is Hastings.'

## --------------------------------------------------------------------------